# TEST ONLY FOR CLASSIFICATION
This code tests the classifier based on data of section CLUSTERING

In [ ]:
import numpy as np
import matplotlib as plt
import tensorflow as tf
from matplotlib.colors import LogNorm
from functions import*
import math
import pandas as pd
import os
from scipy.interpolate import griddata
from matplotlib.colors import LogNorm
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# FUNCTIONS

In [ ]:
def normalize_M_test(M,maxi,mini):
    M_norm = np.zeros((np.shape(M)[0],np.shape(M)[1]))
    for i in range(np.shape(M)[0]):
        for j in range(np.shape(M)[1]):
            M_norm[i,j] = (M[i,j]- mini[j])/(maxi[j]-mini[j])
            #print((M[i,j]- mini[i])/(maxi[i]-mini[i]))
    return M_norm

def get_ma_mi_params(path):
    column = ['max_par','min_par']
    data = pd.read_csv(path,names = column,skiprows=1)
    ma = data.max_par.to_list()
    mi = data.min_par.to_list()
    
    return ma,mi

def accuracy(pred,test):
    return len(np.where((pred-test)==0)[0])/len(test)

# TEST

In [ ]:
from architecture import Classifier

specie = 'vWFs'
dir = '3_cluster'
n_layers_dfnn = 4
n_neurons = 50
n_clusters = 3
N = 5
n_params = 6
path = '../' + specie + '/CLASSIFICATION/'+'/MODELS/'+str(N)+'/'

classifier = Classifier(n_layers = n_layers_dfnn,neurons = n_neurons,length_output = n_clusters)
classifier.built = True
classifier.load_weights(path+dir+'/weights_classifier/classifier_tf')

In [ ]:
param_test = np.loadtxt(open(path+'/Data/'+"param_test.csv", "rb"), delimiter=',')
labels = np.loadtxt(open(path+'/Data/'+"label_test.csv", "rb"), delimiter=',')
ma, mi = get_ma_mi_params(path+'/Data/normalization.csv')
param_test = normalize_M_test(param_test,ma,mi)

In [ ]:
pred = []
for i in param_test:
    i = tf.reshape(i,[-1,n_params])
    out = classifier(i).numpy()
    pred.append(out[0])
classes_pred = []
for i in pred:
    classes_pred.append(np.where(i == np.amax(i))[0][0])
classes_pred = np.array(classes_pred)
labels = np.array(labels)

In [ ]:
acc = accuracy(classes_pred,labels)
plt.figure(figsize = (15,10))
plt.plot(classes_pred,label = 'Predictions',linestyle='dashed')
plt.plot(labels,label = 'Test',linestyle='dashdot')
plt.xlabel('Test Snapshot',fontsize = 16)
plt.ylabel('Class',fontsize = 16)
plt.legend(fontsize = 16)
plt.title(str(n_clusters)+ ' clusters, Accuracy = ' +str(acc), fontsize = 16)
plt.savefig(path+dir'/Figures/predictions.png',dpi=200,facecolor='w',transparent=True,bbox_inches='tight')